# Design goals

- Safe: code validated and executed in a memory-safe sandboxed environment.
- Hardware and platform independant: Can run on many architecture, platforms, and embedded in many environments.
- Compact: small binary format
- modular: program divided into many parts handled separatly
- efficient: can be decoded, validated and compiled rapidly

WASM is virtual ISA (Instruction Set Architecture).  
WASM by itself doesn't define any way to interact with environment. It can be done by calling functions provided by the embeder.

# Concepts

## Values

4 basic types: i32, i64, f32, f64. i32 also used as boolean and memory address.  
No distinction between signed and unsigned, interpret as such depending on instructions.

## Instructions

WASM based on stack machine.  
Sequence of instructions executed in order, that manipulate operands stack.  
Control instructions to alter control flow: expressed with nested block constructs: loop, conditions.

## Traps

Some instructions may produce a trap that abort execution and get caugth by embeder.

## Functions

Code is organized into multiple functions. Each function takes a sequence of values as arguments, and returns a sequence of values.

## Tables

Array of opaque values. For now, can only be used to store function references, and call a function through a pointer.  

## Linear memory

Contiguous mutable array of bytes. Can be extended dynamically.  
Can load/store anywhere on this memory.

## Modules

A WASM binary is a module containing:
- functions
- table
- linear memory
- const and mutable globale variables

## Embedder


Environment to execute and access WASM code

# Semantic Phases

## Decoding

Decode binary into internal representation

## Validation

Performs many checks to make sure module is meaningful and safe (eg type checking)

## Execution

### Instantation

Initialize globals, memories, and tables. Then run module start's function is defined.

### Invocation

Embeder can execute some exported functions of the module.

# Architecture

## Values

numeric values:
- bytes: 0x00 to 0xFF
- integers: signed or unsigned
- floating points: f32 or f64
- name: sequence of chars, defined as scalar by unicode

## Types

### valtype

i32, i64, f32, f64.  
int are signed or unsigned, interpretation depends on instructions.  
f32 and f64 based on IEEE754.

### resulttype

result of executing instruction of block, may be none or one valtype.

### functype

Functions map a vector of valtypes (parameters) to a vector of valtypes (return values).  
Can only return one value for now.

### limits

size range associated to memtype and tabletype

### memtype

Linear memories, classified by their limits range. Can contain data of any type

### tabletype

Table of elements of the same type. For now, only available type is funrec.  
funrec can be any function type signature, so tables can contain heterogeneous functions.

### globaltype

For global variables, const or mutable.

### externtype

Classify import and export types

## Instructions

Code is a sequence of instructions.  
Instructions pop values from operand stack, and then push results to operand stack. For now, can push at most one result.  
Some sequence also have static immediate arguments, usually indices or type annotations.  

Some instructions are structed, and contains nested block of instructions. Used for if/else, loops.

### Expressions

Function bodies and global values are given as expressions. These are a list of instructions finished by end.

## Modules

A wasm file is a module. It contains:
- definitions of types
- definitions of functions
- definitions of tables
- definitions of globals
- defintions of memories
- declare imports
- declare expors

All definitions are index as 0-based indices, without unsigned i32 values.

### Types

a type define a function type. Index needed when declaring function

### Function

A function is defined with a type index, a list of local variables, and a body expression.
local variables is a vector of variables with their type.  
Variables accessed by index in the function body. Locals contains first arguments.

### Tables

Tables define a list of functionref. Right now a module can only have one table, and instructions refers to this table.

### Memories

A memory is a vector of raw unterpreted bytes. In limits, min represents it's initial size, and max the max size it can grows to. Both are units of page size.  
Only one memory can be defined, and all instructions refer to it.  
Memories are initialized through data segments

### Globals

Each global has a type, a name, and an init value (constant expression).  
Globals are referenced through global indices.

### Element Segments

By default a table is unintialized. An element segment define a vector of elements to initialize the subrange of a table.

### Data Segments

By default a memory is 0-inintialized. A data segment defines a vector of raw bytes to initialize a subrange of the memory

### Start Function

start defines the function index of a start function called at module instantiation, after the tables and memories are initialized.


### Exports

Set of exports accessible to the host environment. Can be any kind of definition.

### Imports

Set of imports accessible during instantiation. Can be any kind of definition.

# Validation

Check that module is well-formed before instantiation. Defined by typing rules.